# Load Data

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir("D:\Dropbox\Data-Analysis\Via\project\data")

In [2]:
df = pd.read_csv('green%2Fwk15_sun.csv')
df.head()

,pickup_datetime,trip_time,trip_distance,pickup_long,pickup_lat,dropoff_long,dropoff_lat,passenger_count,hour,dayofweek
0,2015-04-12 00:40:18 UTC,160,0.29,-73.9749,40.6808,-73.9777,40.6846,1,0,1
1,2015-04-12 00:10:47 UTC,210,0.58,-73.9505,40.6716,-73.9397,40.6710,1,0,1
2,2015-04-12 00:49:50 UTC,249,0.90,-73.8299,40.7137,-73.8333,40.7066,1,0,1
3,2015-04-12 00:24:02 UTC,68,0.01,-73.8811,40.7562,-73.8830,40.7559,4,0,1
4,2015-04-12 00:30:10 UTC,248,0.57,-73.9673,40.6836,-73.9607,40.6859,1,0,1


# Geocoder add zipcode and neighborhood

In [3]:
df['pickup_long'] = df['pickup_long'].astype(str)
df['pickup_lat'] = df['pickup_lat'].astype(str)
df['dropoff_long'] = df['dropoff_long'].astype(str)
df['dropoff_lat'] = df['dropoff_lat'].astype(str)
df['pickup_coord']=df[['pickup_lat', 'pickup_long']].apply(lambda x: ",".join(x), axis=1)
df['dropoff_coord']=df[['dropoff_lat', 'dropoff_long']].apply(lambda x: ",".join(x), axis=1)
df.drop(['pickup_long','pickup_lat','dropoff_long','dropoff_lat'],axis=1,inplace=True)
df.head()

,pickup_datetime,trip_time,trip_distance,passenger_count,hour,dayofweek,pickup_coord,dropoff_coord
0,2015-04-12 00:40:18 UTC,160,0.29,1,0,1,"40.6808,-73.9749","40.6846,-73.9777"
1,2015-04-12 00:10:47 UTC,210,0.58,1,0,1,"40.6716,-73.9505","40.671,-73.9397"
2,2015-04-12 00:49:50 UTC,249,0.90,1,0,1,"40.7137,-73.8299","40.7066,-73.8333"
3,2015-04-12 00:24:02 UTC,68,0.01,4,0,1,"40.7562,-73.8811","40.7559,-73.883"
4,2015-04-12 00:30:10 UTC,248,0.57,1,0,1,"40.6836,-73.9673","40.6859,-73.9607"


In [4]:
import sys
sys.path.append('C:\\Python27\\Lib\\site-packages')

In [5]:
from geopy.geocoders import Nominatim
import re
import time
import csv

In [6]:
geolocator = Nominatim()

In [7]:
def get_geocode(coord):
    time.sleep(0.5)
    addr = geolocator.reverse(coord, timeout=60).address
    if addr == None:
        zipcode = 'NaN'
        zone = 'NaN'
        county = 'NaN'
    else:
        strs = [x.strip() for x in addr.split(',')]
        num = str(re.findall(r"\D(\d{5})\D", addr))
        num = str(filter(str.isdigit, num))[:5]
        zipcode = num
        county = strs[-5]
        if strs[-6]=='Manhattan' or strs[-6]=='BK':
            zone = strs[-7]
        else:
            zone = strs[-6]
        #num = str(re.findall(r"\D(\d{5})\D", location.address))
        #num = str(filter(str.isdigit, num))
        #zipcode = num
    return zipcode+','+zone + ',' + county

In [8]:
def get_zipcode(zone):
    return zone.split(',')[0]
def get_neighborhood(zone):
    return zone.split(',')[1]
def get_county(zone):
    return zone.split(',')[2]

In [9]:
df.shape

(59904, 8)

In [10]:
L = len(df)
L

59904

In [11]:
# since df is still too large to run, break it down to smaller pieces and then cat together to a large df

In [12]:
def add_fields(df):
    df['pickup_zone']=df['pickup_coord'].apply(get_geocode)
    df['pickup_zipcode'] = df['pickup_zone'].apply(get_zipcode)
    df['pickup_neighborhood'] = df['pickup_zone'].apply(get_neighborhood)
    df['pickup_county'] = df['pickup_zone'].apply(get_county)
    df['dropoff_zone']=df['dropoff_coord'].apply(get_geocode)
    df['dropoff_zipcode'] = df['dropoff_zone'].apply(get_zipcode)
    df['dropoff_neighborhood'] = df['dropoff_zone'].apply(get_neighborhood)
    df['dropoff_county'] = df['dropoff_zone'].apply(get_county)
    df.drop(['pickup_coord','dropoff_coord','pickup_zone','dropoff_zone'],axis=1,inplace=True)

In [13]:
test = df[:2].copy() #slice by copy, not by view, seeding with 1
test

,pickup_datetime,trip_time,trip_distance,passenger_count,hour,dayofweek,pickup_coord,dropoff_coord
0,2015-04-12 00:40:18 UTC,160,0.29,1,0,1,"40.6808,-73.9749","40.6846,-73.9777"
1,2015-04-12 00:10:47 UTC,210,0.58,1,0,1,"40.6716,-73.9505","40.671,-73.9397"


In [14]:
t0 = time.time()
add_fields(test)
t1 = time.time()-t0
t1

4.670000076293945

In [15]:
test

,pickup_datetime,trip_time,trip_distance,passenger_count,hour,dayofweek,pickup_zipcode,pickup_neighborhood,pickup_county,dropoff_zipcode,dropoff_neighborhood,dropoff_county
0,2015-04-12 00:40:18 UTC,160,0.29,1,0,1,11213,Fort Greene,Kings County,11217,Fort Greene,Kings County
1,2015-04-12 00:10:47 UTC,210,0.58,1,0,1,11216,Crown Heights,Kings County,11213,Eastern Parkway,Kings County


In [16]:
col = test.columns
col

Index([u'pickup_datetime', u'trip_time', u'trip_distance', u'passenger_count',
       u'hour', u'dayofweek', u'pickup_zipcode', u'pickup_neighborhood',
       u'pickup_county', u'dropoff_zipcode', u'dropoff_neighborhood',
       u'dropoff_county'],
      dtype='object')

In [17]:
# store all dataframe slice in dfs
def slice_dataframe(end,step,df):
    dfs=[]
    i = 0
    while i < end:
        if (i+step) > end:
                dfi = df[i:].copy()
        else:
            dfi = df[i:i+step].copy()
        dfs.append(dfi)
        i = i + step
    return dfs

In [18]:
dfs_all = slice_dataframe(L,10000,df)

In [19]:
len(dfs_all)

6

In [26]:
i = 0 
result_df = pd.DataFrame(data=None, columns=col)
while i < L:
    add_fields(dfs_all[i])
    result_df = result_df.append(dfs_all[i],ignore_index = True)
    i = i + 1

# Data cleaning

In [30]:
result_df.dtypes

dayofweek               float64
dropoff_county           object
dropoff_neighborhood     object
dropoff_zipcode         float64
hour                    float64
passenger_count         float64
pickup_county            object
pickup_datetime          object
pickup_neighborhood      object
pickup_zipcode          float64
trip_distance           float64
trip_time               float64
dtype: object

In [33]:
# drop first column if 'Unnamed: 0',
# result_df.drop('Unnamed: 0',axis=1,inplace=True)

In [34]:
# check NA zipcode
result_df['dropoff_zipcode'].isnull().sum()

23

In [35]:
result_df['pickup_zipcode'].isnull().sum()

13

In [36]:
# drop NA, keep index 
result_df.dropna(axis=0, how='any',inplace = True)
result_df.shape

(59868, 12)

In [37]:
# change float to int data type
result_df['dayofweek'] = result_df['dayofweek'].astype(int)
result_df['pickup_zipcode'] = result_df['pickup_zipcode'].astype(int)
result_df['dropoff_zipcode'] = result_df['dropoff_zipcode'].astype(int)
result_df['hour'] = result_df['hour'].astype(int)
result_df['passenger_count'] = result_df['passenger_count'].astype(int)
result_df.head()

,dayofweek,dropoff_county,dropoff_neighborhood,dropoff_zipcode,hour,passenger_count,pickup_county,pickup_datetime,pickup_neighborhood,pickup_zipcode,trip_distance,trip_time
0,1,Kings County,Fort Greene,11217,0,1,Kings County,2015-04-12 00:40:18 UTC,Fort Greene,11213,0.29,160.0
1,1,Kings County,Eastern Parkway,11213,0,1,Kings County,2015-04-12 00:10:47 UTC,Crown Heights,11216,0.58,210.0
2,1,Queens County,Kew Gardens,11415,0,1,Queens County,2015-04-12 00:49:50 UTC,Queens,11415,0.90,249.0
3,1,Queens County,Jackson Heights,11370,0,4,Queens County,2015-04-12 00:24:02 UTC,Jackson Heights,11369,0.01,68.0
4,1,Kings County,Bedford-Stuyvesant,11238,0,1,Kings County,2015-04-12 00:30:10 UTC,Prospect Heights,11238,0.57,248.0


In [38]:
result_df.tail()

,dayofweek,dropoff_county,dropoff_neighborhood,dropoff_zipcode,hour,passenger_count,pickup_county,pickup_datetime,pickup_neighborhood,pickup_zipcode,trip_distance,trip_time
59899,1,Kings County,Prospect Heights,11216,23,1,Kings County,2015-04-12 23:56:29 UTC,Williamsburg,11211,9.17,2494.0
59900,1,Kings County,Williamsburg,11249,23,1,Kings County,2015-04-12 23:17:13 UTC,Boerum Hill,11201,2.81,523.0
59901,1,Bronx County,High Bridge,10452,23,1,New York County,2015-04-12 23:59:20 UTC,Hamilton Heights,10026,3.37,764.0
59902,1,Kings County,Flatlands,11210,23,1,Kings County,2015-04-12 23:50:52 UTC,Fort Greene,11217,5.47,1209.0
59903,1,Queens County,Sunnyside Gardens,11101,23,5,Queens County,2015-04-12 23:05:18 UTC,Jackson Heights,11377,5.09,2014.0


In [39]:
# change weekday to Monday = 1
result_df['dayofweek']=result_df['dayofweek']-1
result_df['dayofweek']= np.where(result_df['dayofweek']== 0,7,result_df['dayofweek'])

In [40]:
result_df

,dayofweek,dropoff_county,dropoff_neighborhood,dropoff_zipcode,hour,passenger_count,pickup_county,pickup_datetime,pickup_neighborhood,pickup_zipcode,trip_distance,trip_time
0,7,Kings County,Fort Greene,11217,0,1,Kings County,2015-04-12 00:40:18 UTC,Fort Greene,11213,0.29,160.0
1,7,Kings County,Eastern Parkway,11213,0,1,Kings County,2015-04-12 00:10:47 UTC,Crown Heights,11216,0.58,210.0
2,7,Queens County,Kew Gardens,11415,0,1,Queens County,2015-04-12 00:49:50 UTC,Queens,11415,0.90,249.0
3,7,Queens County,Jackson Heights,11370,0,4,Queens County,2015-04-12 00:24:02 UTC,Jackson Heights,11369,0.01,68.0
4,7,Kings County,Bedford-Stuyvesant,11238,0,1,Kings County,2015-04-12 00:30:10 UTC,Prospect Heights,11238,0.57,248.0
5,7,Queens County,Jackson Heights,11368,0,1,Queens County,2015-04-12 00:37:20 UTC,Jackson Heights,11369,0.77,245.0
6,7,New York County,Upper West Side,10025,0,1,New York County,2015-04-12 00:54:21 UTC,Morningside Heights,10026,0.40,216.0
7,7,Queens County,Ridgewood,11385,0,1,Kings,2015-04-12 00:06:38 UTC,Bushwick,11237,0.97,355.0
8,7,Kings County,Crown Heights,11238,0,1,Kings County,2015-04-12 00:30:04 UTC,Clinton Hill,11238,1.00,243.0
9,7,New York County,Hamilton Heights,10031,0,1,New York County,2015-04-12 00:03:13 UTC,Harlem,10030,1.07,357.0


In [41]:
# clean zipcode to only include new york da zip code data
nyc_zipcode_df = pd.read_csv('zip_to_zcta10_nyc.csv')
nyc_zipcode = nyc_zipcode_df['ZIP'].values.tolist() #convert to list
nyc_zipcode

[10001L,
 10002L,
 10003L,
 10004L,
 10005L,
 10006L,
 10007L,
 10008L,
 10009L,
 10010L,
 10011L,
 10012L,
 10013L,
 10014L,
 10016L,
 10017L,
 10018L,
 10019L,
 10020L,
 10021L,
 10022L,
 10023L,
 10024L,
 10025L,
 10026L,
 10027L,
 10028L,
 10029L,
 10030L,
 10031L,
 10032L,
 10033L,
 10034L,
 10035L,
 10036L,
 10037L,
 10038L,
 10039L,
 10040L,
 10041L,
 10043L,
 10044L,
 10045L,
 10055L,
 10060L,
 10065L,
 10069L,
 10075L,
 10080L,
 10081L,
 10087L,
 10090L,
 10095L,
 10101L,
 10102L,
 10103L,
 10104L,
 10105L,
 10106L,
 10107L,
 10108L,
 10109L,
 10110L,
 10111L,
 10112L,
 10113L,
 10114L,
 10115L,
 10116L,
 10117L,
 10118L,
 10119L,
 10120L,
 10121L,
 10122L,
 10123L,
 10124L,
 10125L,
 10126L,
 10128L,
 10129L,
 10130L,
 10131L,
 10132L,
 10133L,
 10138L,
 10150L,
 10151L,
 10152L,
 10153L,
 10154L,
 10155L,
 10156L,
 10157L,
 10158L,
 10159L,
 10160L,
 10161L,
 10162L,
 10163L,
 10164L,
 10165L,
 10166L,
 10167L,
 10168L,
 10169L,
 10170L,
 10171L,
 10172L,
 10173L,
 10174L,
 

In [42]:
result_df  = result_df[result_df['pickup_zipcode'].isin(nyc_zipcode)]
result_df  = result_df[result_df['dropoff_zipcode'].isin(nyc_zipcode)]

In [43]:
# make county consistant
result_df['pickup_county'] = np.where(result_df['pickup_county'] == 'Kings','Kings County',result_df['pickup_county'])
result_df['dropoff_county'] = np.where(result_df['dropoff_county'] == 'Kings','Kings County',result_df['dropoff_county'])

In [44]:
result_df.shape

(59276, 12)

In [45]:
result_df.to_csv('green_wk15_sun_zone.csv')

## Geocoder Test Case

In [46]:
c0 = df['dropoff_coord'].iloc[702]
c0

'40.6528,-74.0061'

In [47]:
# check the weird output 
geolocator.reverse(c0).address

u'998, 4th Avenue, Sunset Park, Kings County, NYC, New York, 11232, United States of America'

In [48]:
get_geocode(c0)

u'11232,Sunset Park,Kings County'